In [1]:
import os
import time
import motor.motor_asyncio
import poe_lib
import tqdm
import re
import numpy as np
import datetime

from poe_lib.objects import Note

In [2]:
client = motor.motor_asyncio.AsyncIOMotorClient(os.environ['MONGO_URL'])

In [3]:
def reduce_std(values, weights, n = 1):
    an_array = values
    mean = np.median(an_array)
    standard_deviation = np.std(an_array)
    distance_from_mean = abs(an_array - mean)
    max_deviations = 2
    not_outlier = distance_from_mean < max_deviations * standard_deviation
    values = values[not_outlier]
    weights = weights[not_outlier]
    
    if n > 1:
        values, weights = reduce_std(values, weights, n - 1)
    
    return values, weights

In [4]:
pipeline = [
    {
        '$match': {
            'extended.category': 'currency'
        }
    }, {
        '$project': {
            'typeLine': 1
        }
    }, {
        '$group': {
            '_id': None, 
            'types': {
                '$addToSet': '$typeLine'
            }
        }
    }, {
        '$unwind': {
            'path': '$types'
        }
    }
]
distinct_types = []
async for distinct_type in client.trade.items.aggregate(pipeline):
    distinct_types.append(distinct_type['types'])
    
_ = await client.trade.meta.find_one_and_update(
    {'name': 'currency_types'},
    {'$set': {'name': 'currency_types', 'types': sorted(distinct_types)}},
    upsert = True,
)

In [5]:
currency_types = (await client.trade.meta.find_one({'name': 'currency_types'}))['types']

In [46]:
for currency in currency_types:
    my_filter = {'typeLine': currency, '_price.unit': 'Chaos Orb', '_price.value': {'$exists': True}}
    x = await client.trade.items.count_documents(my_filter)

In [55]:
values = []
my_filter = {'typeLine': 'Rusted Cartography Scarab', '_price.unit': 'Chaos Orb', '_price.value': {'$ne': None}}
async for item in client.trade.items.find(my_filter, projection={'_price.value': 1, 'stackSize': 1}):
    values.append((item['_price']['value'],item['stackSize']))

In [48]:
dtype=[('value', int), ('weight', int)]

In [49]:
values = sorted(values, key=lambda x: x[0])

In [50]:
x = np.array(values)

In [51]:
values = x.T[0]
weights = x.T[1]

In [52]:
values, weights = reduce_std(values, weights, n=10)

In [53]:
print(np.average(values, weights=weights))
print(np.std(values))

2.408893242413462
0.7197421408743286
